In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

# Importing matplotlib to plot images.
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Importing SK-learn to calculate precision and recall
import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score, LeaveOneGroupOut
from sklearn.utils import shuffle
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics.pairwise import euclidean_distances

import pickle as pkl
import h5py

from pathlib import Path
import os.path
import sys
import datetime
import time

In [ ]:
pixel_width = {"N5X":1080,"S3Mini":480,"S4":1080,"N6":1440}
pixel_height = {"N5X":1920,"S3Mini":800,"S4":1920,"N6":2560}

def normalizeWidth(row):
    smartphone = row.Phone
    return row.XPress / pixel_width[smartphone]

def normalizeHeight(row):
    smartphone = row.Phone    
    return row.YPress / pixel_height[smartphone]

In [ ]:
df = pd.read_pickle("../../data/step03.pkl")

In [ ]:
df.XPress = df.apply(lambda x: normalizeWidth(x), axis=1)
df.YPress = df.apply(lambda x: normalizeHeight(x), axis=1)

df.Sensor = df.Sensor.apply(lambda x: x.reshape(-1, 6, 1))

np.random.seed(42)
lst = df.PId.unique()
np.random.shuffle(lst)

shape = df.Sensor.iloc[0].shape

dfTrain = df[df.PId.isin(lst[:14])]
dfTest = df[df.PId.isin(lst[14:])]

train_x = np.concatenate(dfTrain.Sensor.values).reshape(-1, shape[0], shape[1], shape[2])
train_x = train_x.reshape(-1, train_x.shape[1] * train_x.shape[2])
train_y = dfTrain[['XPress', 'YPress']].values

test_x = np.concatenate(dfTest.Sensor.values).reshape(-1, shape[0], shape[1], shape[2])
test_x = test_x.reshape(-1, test_x.shape[1] * test_x.shape[2])
test_y = dfTest[['XPress', 'YPress']].values

# RandomForestRegressor

In [ ]:
regressor = RandomForestRegressor()
params = {'max_depth':range(3,12,2)}
clf = GridSearchCV(regressor, params, cv=5, n_jobs = 7, verbose=1)
clf.fit(train_x, train_y)

In [ ]:
results = clf.best_estimator_.predict(test_x)
print("RandomForestRegressor", np.mean(euclidean_distances(test_y, results)))

# DecisionTreeRegressor

In [ ]:
regressor = DecisionTreeRegressor()
params = {'max_depth':range(3,12,3)}
clf = GridSearchCV(regressor, params, cv=5, n_jobs = 7, verbose=1)
clf.fit(train_x, train_y)

In [ ]:
results = clf.best_estimator_.predict(test_x)
print("DecisionTreeRegressor", np.mean(euclidean_distances(test_y, results)))

# KNeighborsRegressor

In [ ]:
regressor = KNeighborsRegressor()
params = {'n_neighbors':range(3,6,2)}
clf = GridSearchCV(regressor, params, cv=5, n_jobs = 7, verbose=1)
clf.fit(train_x, train_y)

In [ ]:
results = clf.best_estimator_.predict(test_x)
print("KNeighborsRegressor", np.mean(euclidean_distances(test_y, results)))

# MLPRegressor

In [ ]:
regressor = MLPRegressor(hidden_layer_sizes = (40,20))
params = {'hidden_layer_sizes':range(10,150,20)}
clf = GridSearchCV(regressor, params, cv=5, n_jobs = 7, verbose=1)
clf.fit(train_x, train_y)

In [ ]:
results = clf.best_estimator_.predict(test_x)
print("MLPRegressor", np.mean(euclidean_distances(test_y, results)))

# single phones

In [ ]:
for smartphone, constant_pixels in zip(["S3Mini", "S4", "N5X", "N6"], [0.1089, 0.0577, 0.06, 0.05109]):
    df = pd.read_pickle("../../data/step03.pkl")
    
    df = df[(df.Phone == smartphone) ]
    df.Sensor = df.Sensor.apply(lambda x: x.reshape(-1, 6, 1))

    np.random.seed(42)
    lst = df.PId.unique()
    np.random.shuffle(lst)

    shape = df.Sensor.iloc[0].shape

    dfTrain = df[df.PId.isin(lst[:14])]
    dfTest = df[df.PId.isin(lst[14:])]

    train_x = np.concatenate(dfTrain.Sensor.values).reshape(-1, shape[0], shape[1], shape[2])
    train_x = train_x.reshape(-1, train_x.shape[1] * train_x.shape[2])
    train_y = dfTrain[['XPress', 'YPress']].values

    test_x = np.concatenate(dfTest.Sensor.values).reshape(-1, shape[0], shape[1], shape[2])
    test_x = test_x.reshape(-1, test_x.shape[1] * test_x.shape[2])
    test_y = dfTest[['XPress', 'YPress']].values
    
    regressor = RandomForestRegressor()
    params = {'max_depth':range(3,12,2)}
    clf = GridSearchCV(regressor, params, cv=5, n_jobs = 7, verbose=1)
    clf.fit(train_x, train_y)
    results = clf.best_estimator_.predict(test_x)
    print(smartphone, "RandomForestRegressor", np.mean(euclidean_distances(test_y, results)) * constant_pixels, "mm")
    
    regressor = DecisionTreeRegressor()
    params = {'max_depth':range(3,12,3)}
    clf = GridSearchCV(regressor, params, cv=5, n_jobs = 7, verbose=1)
    clf.fit(train_x, train_y)
    results = clf.best_estimator_.predict(test_x)
    print(smartphone, "DecisionTreeRegressor", np.mean(euclidean_distances(test_y, results)) * constant_pixels, "mm")
    
    regressor = KNeighborsRegressor()
    params = {'n_neighbors':range(3,6,2)}
    clf = GridSearchCV(regressor, params, cv=5, n_jobs = 7, verbose=1)
    clf.fit(train_x, train_y)
    results = clf.best_estimator_.predict(test_x)
    print(smartphone, "KNeighborsRegressor", np.mean(euclidean_distances(test_y, results)) * constant_pixels, "mm")
    
    regressor = MLPRegressor(hidden_layer_sizes = (40,20))
    params = {'hidden_layer_sizes':range(10,150,20)}
    clf = GridSearchCV(regressor, params, cv=5, n_jobs = 7, verbose=1)
    clf.fit(train_x, train_y)
    results = clf.best_estimator_.predict(test_x)
    print(smartphone, "MLPRegressor", np.mean(euclidean_distances(test_y, results)) * constant_pixels, "mm")

# Plotting

In [ ]:
fig,ax = plt.subplots(figsize = (3,6))
ax.set_xlim(0,1080)
ax.set_ylim(1920,0)
ax.scatter(test_y[:,0],test_y[:,1],alpha = 0.3)
ax.scatter(results[:,0],results[:,1],alpha = 0.3)